In [90]:
import keras
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense, Dropout, Activation
from keras.models import Sequential
import numpy as np
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [121]:
%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [122]:
train_data = pd.read_csv('text_emotion.csv')
train_data.head()

,sentiment,content
0,joy,[ On days when I feel close to my partner and ...
1,fear,Every time I imagine that someone I love or I ...
2,anger,When I had been obviously unjustly treated and...
3,sadness,When I think about the short time that we live...
4,disgust,At a gathering I found myself involuntarily si...


In [125]:
train_data['target'] = train_data.sentiment.astype('category').cat.codes

In [126]:
MAX_LENGTH = 100
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data.content.values)
post_seq = tokenizer.texts_to_sequences(train_data.content.values)
post_seq_padded = pad_sequences(post_seq, maxlen=MAX_LENGTH)

In [133]:
x_train, x_test, y_train, y_test = train_test_split(post_seq, train_data.target, test_size=0.05)

In [134]:
num_classes = max(y_train) + 1
tokenizer = Tokenizer(num_words=MAX_LENGTH)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [150]:
model = Sequential()
model.add(Dense(512, input_shape=(MAX_LENGTH,)))
model.add(Dense(256, input_shape=(MAX_LENGTH/2,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.metrics_names)

['loss', 'acc']


In [151]:
batch_size = 64
epochs = 5

In [152]:
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0.1)
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 6542 samples, validate on 727 samples
Epoch 1/5
6542/6542 [==============================] - 1s 157us/step - loss: 1.8235 - acc: 0.2791 - val_loss: 1.7121 - val_acc: 0.3287
Epoch 2/5
6542/6542 [==============================] - 0s 61us/step - loss: 1.6739 - acc: 0.3565 - val_loss: 1.7016 - val_acc: 0.3411
Epoch 3/5
6542/6542 [==============================] - 0s 63us/step - loss: 1.6239 - acc: 0.3844 - val_loss: 1.6828 - val_acc: 0.3439
Epoch 4/5
6542/6542 [==============================] - 0s 62us/step - loss: 1.5910 - acc: 0.3961 - val_loss: 1.6818 - val_acc: 0.3453
Epoch 5/5
383/383 [==============================] - 0s 29us/step
Test loss: 1.6000489239281837
Test accuracy: 0.4020887740131458
